In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
from collections import Counter

%matplotlib inline

In [2]:
def resample_points(pos_filename, percentage):
    in_filename = "/Users/forero/github/cosmo_datasets/abacus/"+pos_filename
    out_filename = "/Users/forero/github/cosmo_datasets/abacus/"+"p_{:03d}_".format(percentage)+pos_filename
    print(in_filename)
    print(out_filename)
    data = np.loadtxt(in_filename)
    r = np.random.random(len(data))
    sample_data = data[r<percentage/100]
    print(len(sample_data), len(data))
    np.savetxt(out_filename, sample_data)
    return  "p_{:03d}_".format(percentage)+pos_filename
    
    
def run_beta_skel(pos_filename, beta=1.0):
    skel = "./LSS_BSK_calc "
    in_filename = "/Users/forero/github/cosmo_datasets/abacus/"+pos_filename
    out_filename =  "beta_{:.1f}_{}".format(beta, pos_filename)
    index_filename = "xdl_beta_skeleton/{}.BSKIndex".format(out_filename)
    if not os.path.exists(index_filename):
        cmd = "./LSS_BSK_calc -beta {} -numNNB {} -input {}  -output {}".format(beta, 300, in_filename, out_filename)
        print(cmd)
        os.system(cmd)
    return index_filename, in_filename


def get_entropy_beta_skel(in_filename, index_filename):
    beta_data = np.int_(np.loadtxt(index_filename))
    point_data = np.loadtxt(in_filename)
    
    # remove the duplicate points using the fact that the first column is ordered in ngl-beta
    ii = beta_data[:,0]<beta_data[:,1]
    beta_data = beta_data[ii]

    n_points = len(point_data)
    #print(n_points)

    # Flattend and count links
    beta_data_flat = beta_data.flatten()
    beta_link_count = Counter(Counter(beta_data_flat).values())

    # Count how many points have zero links
    unique_beta_id = len(set(beta_data_flat))
    if (n_points - unique_beta_id)>0:
        beta_link_count[0] = n_points - unique_beta_id

    assert np.sum(list(beta_link_count.values()))==n_points
    # compute probability array
    proba = []
    total_link = np.sum(list(beta_link_count.values()))
    for k in beta_link_count:
        #print(k, beta_link_count[k]/total_link)
        proba.append(beta_link_count[k]/total_link)
    proba = np.array(proba)
    print(proba, proba.sum())
    entropy = np.sum(-proba*np.log10(proba)/np.log10(2))
    print('Entropy', entropy)
    return entropy

In [5]:
resample_points("shell_00_stdcosmo_norsd_z0.100.dat", 5)

/Users/forero/github/cosmo_datasets/abacus/shell_00_stdcosmo_norsd_z0.100.dat
/Users/forero/github/cosmo_datasets/abacus/p_005_shell_00_stdcosmo_norsd_z0.100.dat
1749 34802


'p_005_shell_00_stdcosmo_norsd_z0.100.dat'

In [3]:
pos_names = []
for p  in np.arange(10,110,10):
    pos_names.append(resample_points("sphere_00_stdcosmo_norsd_z0.100.dat", p))

/Users/forero/github/cosmo_datasets/abacus/sphere_00_stdcosmo_norsd_z0.100.dat
/Users/forero/github/cosmo_datasets/abacus/p_010_sphere_00_stdcosmo_norsd_z0.100.dat
8223 83216
/Users/forero/github/cosmo_datasets/abacus/sphere_00_stdcosmo_norsd_z0.100.dat
/Users/forero/github/cosmo_datasets/abacus/p_020_sphere_00_stdcosmo_norsd_z0.100.dat
16530 83216
/Users/forero/github/cosmo_datasets/abacus/sphere_00_stdcosmo_norsd_z0.100.dat
/Users/forero/github/cosmo_datasets/abacus/p_030_sphere_00_stdcosmo_norsd_z0.100.dat
24763 83216
/Users/forero/github/cosmo_datasets/abacus/sphere_00_stdcosmo_norsd_z0.100.dat
/Users/forero/github/cosmo_datasets/abacus/p_040_sphere_00_stdcosmo_norsd_z0.100.dat
33196 83216
/Users/forero/github/cosmo_datasets/abacus/sphere_00_stdcosmo_norsd_z0.100.dat
/Users/forero/github/cosmo_datasets/abacus/p_050_sphere_00_stdcosmo_norsd_z0.100.dat
41976 83216
/Users/forero/github/cosmo_datasets/abacus/sphere_00_stdcosmo_norsd_z0.100.dat
/Users/forero/github/cosmo_datasets/abacus

In [33]:
pos_names = []
for p  in np.arange(10,110,10):
    pos_names.append(resample_points("random_sphere_00_stdcosmo_norsd_z0.100.dat", p))

/Users/forero/github/cosmo_datasets/abacus/random_sphere_00_stdcosmo_norsd_z0.100.dat
/Users/forero/github/cosmo_datasets/abacus/p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat
8265 83216
/Users/forero/github/cosmo_datasets/abacus/random_sphere_00_stdcosmo_norsd_z0.100.dat
/Users/forero/github/cosmo_datasets/abacus/p_020_random_sphere_00_stdcosmo_norsd_z0.100.dat
16667 83216
/Users/forero/github/cosmo_datasets/abacus/random_sphere_00_stdcosmo_norsd_z0.100.dat
/Users/forero/github/cosmo_datasets/abacus/p_030_random_sphere_00_stdcosmo_norsd_z0.100.dat
24915 83216
/Users/forero/github/cosmo_datasets/abacus/random_sphere_00_stdcosmo_norsd_z0.100.dat
/Users/forero/github/cosmo_datasets/abacus/p_040_random_sphere_00_stdcosmo_norsd_z0.100.dat
33320 83216
/Users/forero/github/cosmo_datasets/abacus/random_sphere_00_stdcosmo_norsd_z0.100.dat
/Users/forero/github/cosmo_datasets/abacus/p_050_random_sphere_00_stdcosmo_norsd_z0.100.dat
41537 83216
/Users/forero/github/cosmo_datasets/abacus/random_s

In [34]:
beta_values = np.arange(1.0, 5.5, 0.5)
p_values = np.arange(10,110,10)
print(beta_values)
index_files = {}
in_files = {}

for i in beta_values:
    index_files[i] = {}
    in_files[i] = {}

for pos_name,p_value in zip(pos_names, p_values):
    for i in beta_values:
        print(pos_name, i)
        index_files[i][p_value], in_files[i][p_value] = run_beta_skel(pos_name, beta=i)

[1.  1.5 2.  2.5 3.  3.5 4.  4.5 5. ]
p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat 1.0
./LSS_BSK_calc -beta 1.0 -numNNB 300 -input /Users/forero/github/cosmo_datasets/abacus/p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat  -output beta_1.0_p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat
p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat 1.5
./LSS_BSK_calc -beta 1.5 -numNNB 300 -input /Users/forero/github/cosmo_datasets/abacus/p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat  -output beta_1.5_p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat
p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat 2.0
./LSS_BSK_calc -beta 2.0 -numNNB 300 -input /Users/forero/github/cosmo_datasets/abacus/p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat  -output beta_2.0_p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat
p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat 2.5
./LSS_BSK_calc -beta 2.5 -numNNB 300 -input /Users/forero/github/cosmo_datasets/abacus/p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat  -out

In [35]:
entropy_values = {}
for i in beta_values:
    entropy_values[i] = []
    
for pos_name,p_value in zip(pos_names, p_values):
    for i in beta_values:
        print(pos_name, i)
        entropy_values[i].append(get_entropy_beta_skel(in_files[i][p_value], index_files[i][p_value]))

p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat 1.0
[1.10586812e-01 1.60677556e-01 1.28372656e-01 1.75438596e-01
 8.89292196e-02 1.63460375e-01 4.81548699e-02 1.28251664e-02
 5.54143981e-02 2.85541440e-02 1.69388990e-02 1.93587417e-03
 2.17785844e-03 4.96067756e-03 8.46944949e-04 3.62976407e-04
 1.20992136e-04 2.41984271e-04] 1.0
Entropy 3.1941492781278122
p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat 1.5
[0.3246219  0.26751361 0.11820932 0.03726558 0.0091954  0.19346642
 0.00193587 0.04476709 0.00036298 0.00266183] 1.0
Entropy 2.342502266149818
p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat 2.0
[0.26073805 0.46448881 0.20967937 0.04573503 0.01802783 0.00133091] 0.9999999999999999
Entropy 1.812784050645408
p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat 2.5
[0.3707199  0.12607381 0.39491833 0.09364791 0.00556564 0.00907441] 1.0
Entropy 1.8599271808731035
p_010_random_sphere_00_stdcosmo_norsd_z0.100.dat 3.0
[0.24319419 0.2492438  0.43303085 0.03315185 0.00072595 0.04065336] 1.0
E

In [36]:
all_values = np.ones((len(beta_values), len(p_values)))
print(all_values)
for i, beta in enumerate(beta_values):
    a = np.array(entropy_values[beta])
    print(i)
    all_values[i] = a

np.savetxt('entropias_por_random.txt', all_values, fmt='%.10e')

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
0
1
2
3
4
5
6
7
8
